## SILVER TO GOLD LAYER


In [9]:
import pandas as pd
from common_utilities import Portfolio, global_path

### Gold Layer - Trade History


In [10]:
# read the csv file
df_TradeHistory = pd.read_csv(global_path.tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by="datetime")

In [11]:
portfolio = Portfolio()

# Apply the function of trade logic to each row of the DataFrame
data = [portfolio.trade(row.to_dict()) for _, row in df_TradeHistory.iterrows()]

df_TradeHistory = pd.DataFrame(data)

# Select columns that end with 'price' or 'amount'
columns_to_round = [
    col
    for col in df_TradeHistory.columns
    if col.endswith("price") or col.endswith("amount")
]

# Round the values in the selected columns to two decimal places
df_TradeHistory[columns_to_round] = df_TradeHistory[columns_to_round].round(2)

In [12]:
# df_TradeHistory = df_TradeHistory[
#     [
#         "datetime",
#         "exchange",
#         "segment",
#         "stock_name",
#         "side",
#         "buy_quantity",
#         "buy_price",
#         "buy_amount",
#         "sell_quantity",
#         "sell_price",
#         "sell_amount",
#         "holding_quantity",
#         "avg_price",
#         "holding_amount",
#         "pnl_amount"
#     ]
# ]

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by=["segment", "stock_name", "datetime"])

# Save the result as a csv file
df_TradeHistory.to_csv(global_path.tradehistory_gold_file_path, index=None)
df_TradeHistory.info()
print("GOLD Layer csv file for trade history successfully created at:")
print(global_path.tradehistory_gold_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 175 entries, 2 to 93
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          175 non-null    datetime64[ns]
 1   exchange          175 non-null    object        
 2   segment           175 non-null    object        
 3   stock_name        175 non-null    object        
 4   scrip_code        175 non-null    object        
 5   side              175 non-null    object        
 6   amount            175 non-null    float64       
 7   quantity          175 non-null    float64       
 8   price             175 non-null    float64       
 9   avg_price         175 non-null    float64       
 10  holding_quantity  175 non-null    int64         
 11  holding_amount    175 non-null    float64       
 12  pnl_amount        175 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 19.1+ KB
GOLD Laye